<a href="https://colab.research.google.com/github/RMottram/2D-CNN/blob/master/RF_SVM_COLAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi

Tue Jan 18 13:14:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import glob

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!unzip /content/drive/MyDrive/Colab\ Notebooks/64x64_V2.zip > /dev/null

In [58]:
train_normal_loc = '/content/64x64_V2/80-20/row-wise/train_normal'
test_normal_loc = '/content/64x64_V2/80-20/row-wise/test_normal'
train_ip_loc = '/content/64x64_V2/80-20/row-wise/train_ip'
test_ip_loc = '/content/64x64_V2/80-20/row-wise/test_ip'
train_port_loc = '/content/64x64_V2/80-20/row-wise/train_port'
test_port_loc = '/content/64x64_V2/80-20/row-wise/test_port'

In [59]:
train_normal_labels = []
test_normal_labels = []
train_ip_labels = []
test_ip_labels = []
train_port_labels = []
test_port_labels = []

# looping the directory
for i in glob.glob('/content/64x64_V2/80-20/row-wise/train_normal/train_normal_equ_*.jpg', recursive=True):
    train_normal_labels.append(0)
    
for j in glob.glob('/content/64x64_V2/80-20/row-wise/test_normal/test_normal_equ_*.jpg', recursive=True):
    test_normal_labels.append(0)

for k in glob.glob('/content/64x64_V2/80-20/row-wise/train_ip/train_ip_equ_*.jpg', recursive=True):
    train_ip_labels.append(1)

for l in glob.glob('/content/64x64_V2/80-20/row-wise/test_ip/test_ip_equ_*.jpg', recursive=True):
    test_ip_labels.append(1)

for m in glob.glob('/content/64x64_V2/80-20/row-wise/train_port/train_port_equ_*.jpg', recursive=True):
    train_port_labels.append(2)

for n in glob.glob('/content/64x64_V2/80-20/row-wise/test_port/test_port_equ_*.jpg', recursive=True):
    test_port_labels.append(2)


In [60]:
print(np.shape(train_normal_labels))
print(np.shape(test_normal_labels))
print(np.shape(train_ip_labels))
print(np.shape(test_ip_labels))
print(np.shape(train_port_labels))
print(np.shape(test_port_labels))

(1714,)
(429,)
(5437,)
(1360,)
(1779,)
(445,)


In [79]:
training_imgs = []
testing_imgs = []

In [81]:
train_img_path = '/content/64x64_V2/80-20/row-wise/train_ip/train_ip_equ_*.jpg'

files = glob.glob(train_img_path)

for my_file in files:
    image = cv2.imread(my_file)
    training_imgs.append(image)

In [83]:
test_img_path = '/content/64x64_V2/80-20/row-wise/test_ip/test_ip_equ_*.jpg'

files = glob.glob(test_img_path)

for my_file in files:
    image = cv2.imread(my_file)
    testing_imgs.append(image)

In [84]:
training_imgs = np.array(training_imgs, dtype = 'float32')
testing_imgs = np.array(testing_imgs, dtype = 'float32')

training_imgs.shape

(7151, 64, 64, 3)

In [85]:
training_imgs = training_imgs.reshape(-1, 12288)
testing_imgs = testing_imgs.reshape(-1, 12288)

In [86]:
training_labels = train_normal_labels + train_ip_labels 
testing_labels = test_normal_labels + test_ip_labels

np.shape(training_labels)
# np.shape(testing_labels)

(7151,)

In [87]:
import sklearn
from sklearn.utils import shuffle
training_imgs, training_labels = sklearn.utils.shuffle(training_imgs, training_labels, random_state=42)
testing_imgs, testing_labels = sklearn.utils.shuffle(testing_imgs, testing_labels, random_state=42)

In [39]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)

rf.fit(training_imgs, training_labels)

RandomForestClassifier()

In [40]:
print(f'Training Score - {round(rf.score(training_imgs, training_labels), 5) * 100}%')
print(f'Testing score - {round(rf.score(testing_imgs, testing_labels), 5) * 100}%')

Training Score - 72.665%
Testing score - 60.877%


# SVM

In [88]:
from sklearn.svm import SVC

svm = SVC(kernel='sigmoid')
svm.fit(training_imgs, training_labels)

print(f'Training Score - {round(svm.score(training_imgs, training_labels), 5) * 100}%')
print(f'Testing score - {round(svm.score(testing_imgs, testing_labels), 5) * 100}%')

Training Score - 95.231%
Testing score - 94.96900000000001%
